# **Sript für die Ausführung von MBUS-Prep**

Mit diesem Script werden die JSON-Files der MBus-Rohdaten aufgelöst und in einem SQL-Table abgelegt.

In [1]:
 /****** Object:  Table [dbo].[MBUS_Values]    Script Date: 03.03.2021 13:19:16 ******/
DROP TABLE [dbo].[MBUS_Values]
DROP TABLE [dbo].[MBUS_Meta]
DROP TABLE [dbo].[MBUS_Data]
GO

/****** Object:  Table [dbo].[MBUS_Values]    Script Date: 03.03.2021 13:19:16 ******/
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO



SELECT        mbusmaster + '_' + measurementSerieId AS JoinID, dateadd(S, CONVERT(bigint, effectivedate), '1970-01-01') AS datetime, value, valueAsString
INTO          MBUS_Data
FROM          dbo.Wärmezählerdaten CROSS APPLY OPENJSON(json) WITH (measurementSerieId varchar(5), effectiveDate varchar(10), value varchar(20), valueAsString varchar (20))

SELECT        distinct (mbusmaster + '_' + measurementSerieId) AS JoinID, sourceId AS meterID, sourcePosition AS Zählerbeschr, unitString AS unit, unitTypeId AS id, deviceTypeString
INTO          MBUS_Meta
FROM          dbo.MdmSerie CROSS APPLY OPENJSON(json) WITH (sourceId varchar(10), unitString varchar(10), sourcePosition varchar(100), unitTypeId varchar(10), measurementSerieId varchar(5), deviceTypeString varchar(10))
GO

SELECT        dbo.MBUS_Data.datetime, dbo.MBUS_Meta.meterID, dbo.MBUS_Meta.id, dbo.MBUS_Meta.unit, dbo.MBUS_Data.value, dbo.MBUS_Data.valueAsString, dbo.MBUS_Meta.deviceTypeString
INTO          MBUS_Values
FROM          dbo.MBUS_Data 
LEFT JOIN     dbo.MBUS_Meta ON dbo.MBUS_Data.JoinID = dbo.MBUS_Meta.JoinID
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(80991038 rows affected)

(10786 rows affected)

(80991038 rows affected)

Total execution time: 00:25:27.983

# **Mbus-Master Kontrolle**

In [3]:
SELECT mbusmaster,  APPROX_COUNT_DISTINCT(meterID)
  FROM [dbo].[MBUS_Values_neu]
  WHERE DATEPART(YEAR, [datetime]) = DATEPART(YEAR, GETDATE()) AND
        DATEPART(WEEK,[datetime])>(DATEPART(WEEK, GETDATE())-2)
group by mbusmaster
 

Warning: Null value is eliminated by an aggregate or other SET operation.

(22 rows affected)

Total execution time: 00:01:33.875

mbusmaster,(No column name)
NULL,0
10.123.4.41:80,32
10.21.1.41:80,128
10.24.8.41:80,72
10.26.1.41:80,66
10.33.2.41:80,55
10.33.3.41:80,9
10.33.4.41:80,77
10.39.1.31:80,110
10.39.2.41:80,68
